In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import copy

from rosbags.rosbag2 import Reader
from rosbags.serde import deserialize_cdr
from rosbags.typesys import get_types_from_idl, register_types

from pathlib import Path


import tvm
from tvm.contrib import graph_executor
from tvm.contrib import graph_runtime

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
import sys

sys.path.append('/opt/ros/humble/local/lib/python3.10/dist-packages')
sys.path.append('/opt/ros/humble/lib/python3.10/site-packages')

#print(sys.path)
print(os.environ['LD_LIBRARY_PATH'])

/home/tohmae/.pyenv/versions/miniconda3-4.7.12/envs/py310tvm/lib/python3.10/site-packages/tvm/:/opt/ros/humble/lib


In [3]:
pcd_filename = "pointcloud/test_pointcloud1.pcd"
timelag = 0.0

In [4]:
pc = o3d.io.read_point_cloud(pcd_filename)
pc_msg = np.asarray(pc.points)

In [5]:
pc_msg.shape

(221733, 3)

In [6]:
class Config:
    point_cloud_range = [-89.6, -89.6, -3.0, 89.6, 89.6, 5.0]
    voxel_size = [0.32, 0.32, 8.0]
    point_feature_size = 4
    point_dim_size = 3
    max_voxel_size = 40000
    max_point_in_voxel_size = 32
    score_threshold = 0.35
    circle_nms_dist_threshold = 1.5
    yaw_norm_threshold = 0.0
    downsample_factor_ = 1
    circle_nms_dist_threshold = 1.5
    class_names = ["CAR", "PEDESTRIAN", "BICYCLE"]
    rename_car_to_truck_and_bus = True
    has_twist = False
    
    
    range_min_x_ = point_cloud_range[0]
    range_min_y_ = point_cloud_range[1]
    range_min_z_ = point_cloud_range[2]
    range_max_x_ = point_cloud_range[3]
    range_max_y_ = point_cloud_range[4]
    range_max_z_ = point_cloud_range[5]

    voxel_size_x_ = voxel_size[0]
    voxel_size_y_ = voxel_size[1]
    voxel_size_z_ = voxel_size[2]

    grid_size_x_ = int((range_max_x_ - range_min_x_) / voxel_size_x_)
    grid_size_y_ = int((range_max_y_ - range_min_y_) / voxel_size_y_)
    grid_size_z_ = int((range_max_z_ - range_min_z_) / voxel_size_z_)
    
    offset_x_ = range_min_x_ + voxel_size_x_ / 2
    offset_y_ = range_min_y_ + voxel_size_y_ / 2
    offset_z_ = range_min_z_ + voxel_size_z_ / 2
    
    down_grid_size_x_ = int(grid_size_x_ / downsample_factor_)
    down_grid_size_y_ = int(grid_size_y_ / downsample_factor_)

    range_ = np.asarray((range_min_x_, range_min_y_, range_min_z_, range_max_x_, range_max_y_, range_max_z_))
    grid_size_ = np.asarray((grid_size_x_, grid_size_y_, grid_size_z_))
    recip_voxel_size_ = np.asarray((1.0 / voxel_size_x_, 1.0 / voxel_size_y_, 1.0 / voxel_size_z_))

In [7]:
config = Config()

In [8]:
config.grid_size_x_, config.range_min_x_

(560, -89.6)

In [9]:
config.range_, config.grid_size_, config.recip_voxel_size_, config.down_grid_size_x_, config.down_grid_size_y_

(array([-89.6, -89.6,  -3. ,  89.6,  89.6,   5. ]),
 array([560, 560,   1]),
 array([3.125, 3.125, 0.125]),
 560,
 560)

In [10]:
config.voxel_size_x_, config.voxel_size_y_, config.voxel_size_z_, config.offset_x_, config.offset_y_, config.offset_z_

(0.32, 0.32, 8.0, -89.44, -89.44, 1.0)

In [11]:
def pointsToVoxel(pc_msg):
    grid_size = config.grid_size_z_ * config.grid_size_y_ * config.grid_size_x_
    coord_to_voxel_idx = np.repeat(-1, grid_size)

    df_pc = pd.DataFrame(pc_msg,columns=['x','y','z'])
    df_pc['timelag'] = timelag
    df_pc['coord_x'] = ((df_pc['x'] - config.range_[0]) * config.recip_voxel_size_[0]).astype('int32')
    df_pc['coord_y'] = ((df_pc['y'] - config.range_[1]) * config.recip_voxel_size_[1]).astype('int32')
    df_pc['coord_z'] = ((df_pc['z'] - config.range_[2]) * config.recip_voxel_size_[2]).astype('int32')
    
    df_pc = df_pc[(df_pc['coord_x']>=0) & (df_pc['coord_x'] < config.grid_size_[0])]
    df_pc = df_pc[(df_pc['coord_y']>=0) & (df_pc['coord_y'] < config.grid_size_[1])]
    df_pc = df_pc[(df_pc['coord_z']>=0) & (df_pc['coord_z'] < config.grid_size_[2])]
    
    df_pc['coord_idx'] = df_pc['coord_z'] * config.grid_size_y_ * config.grid_size_x_ +df_pc['coord_y'] * config.grid_size_x_ + df_pc['coord_x']
    
    np.random.seed(seed=32)
    df_pc['random'] = np.random.rand(len(df_pc))
    df_pc['rank'] = df_pc.groupby(['coord_idx'])['random'].rank(method='first')
    df_pc = df_pc[df_pc['rank']<=32]
    
    df_num_points_per_voxel = df_pc['coord_idx'].value_counts().reset_index()
    df_num_points_per_voxel.columns = ['coord_idx', 'point_cnt']
    df_num_points_per_voxel['voxel_idx'] = df_num_points_per_voxel.index
    df_num_points_per_voxel = df_num_points_per_voxel[['voxel_idx','coord_idx','point_cnt']]
    voxel_cnt = len(df_num_points_per_voxel)
    
    df_pc = pd.merge(df_pc, df_num_points_per_voxel, how='inner', on='coord_idx')
    df_coordinate_tmp = df_pc[~df_pc.duplicated(subset='voxel_idx')].sort_values('voxel_idx')[['voxel_idx', 'coord_z','coord_y', 'coord_x']].reset_index(drop=True)

    df_coordinate = pd.DataFrame(np.arange(0, config.max_voxel_size),columns=['voxel_idx'])
    df_coordinate = pd.merge(df_coordinate, df_coordinate_tmp, how='left', on='voxel_idx')
    df_coordinate = df_coordinate.fillna(-1)
    df_coordinate = df_coordinate.astype('int32')
    
    df_voxel = pd.DataFrame(np.repeat(np.arange(0,config.max_voxel_size), config.max_point_in_voxel_size),columns=['voxel_idx'])
    df_voxel['rank'] = np.tile(np.arange(1,config.max_point_in_voxel_size+1), config.max_voxel_size)
    df_voxel = pd.merge(df_voxel, df_pc, how='left', on=['voxel_idx', 'rank'])
    nancols = ['x','y','z','timelag']
    for nancol in nancols:
        df_voxel[nancol] = df_voxel[nancol].fillna(0)
    df_voxel = df_voxel[['voxel_idx','rank','x','y','z','timelag']]
    
    return df_pc, df_voxel, df_coordinate, df_num_points_per_voxel, voxel_cnt

In [12]:
df_pc, df_voxel, df_coordinate, df_num_points_per_voxel,voxel_cnt = pointsToVoxel(pc_msg)

In [13]:
len(df_pc), len(df_voxel), len(df_coordinate), len(df_num_points_per_voxel), voxel_cnt

(157025, 1280000, 40000, 31227, 31227)

In [14]:
df_coordinate.head(), df_coordinate.tail()

(   voxel_idx  coord_z  coord_y  coord_x
 0          0        0      293      282
 1          1        0      260      277
 2          2        0      324      432
 3          3        0      278      354
 4          4        0      242      288,
        voxel_idx  coord_z  coord_y  coord_x
 39995      39995       -1       -1       -1
 39996      39996       -1       -1       -1
 39997      39997       -1       -1       -1
 39998      39998       -1       -1       -1
 39999      39999       -1       -1       -1)

In [15]:
def generatefeature(df_pc):
    df_pc_mean = df_pc[['x','y','z', 'voxel_idx']].groupby('voxel_idx').mean().reset_index()
    df_pc_mean.columns = ['voxel_idx','x_mean', 'y_mean', 'z_mean']
    
    df_pc['x_offset'] = df_pc['coord_x'] * config.voxel_size_x_ + config.offset_x_
    df_pc['y_offset'] = df_pc['coord_y'] * config.voxel_size_y_ + config.offset_y_
    
    df_feature = pd.merge(df_pc, df_pc_mean, on='voxel_idx')
    
    df_feature['feature_0'] = df_feature['x']
    df_feature['feature_1'] = df_feature['y']
    df_feature['feature_2'] = df_feature['z']
    df_feature['feature_3'] = df_feature['timelag']

    df_feature['feature_4'] = df_feature['x'] - df_feature['x_mean']
    df_feature['feature_5'] = df_feature['y'] - df_feature['y_mean']
    df_feature['feature_6'] = df_feature['z'] - df_feature['z_mean']

    df_feature['feature_7'] = df_feature['x'] - df_feature['x_offset']
    df_feature['feature_8'] = df_feature['y'] - df_feature['y_offset']
    
    df_feature_new = pd.DataFrame(np.repeat(np.arange(0,config.max_voxel_size), config.max_point_in_voxel_size),columns=['voxel_idx'])
    df_feature_new['rank'] = np.tile(np.arange(1,config.max_point_in_voxel_size+1), config.max_voxel_size)
    df_feature_new = pd.merge(df_feature_new, df_feature, how='left', on=['voxel_idx', 'rank'])

    feature_cols = ['feature_{}'.format(i) for i in range(9)]
    for nancol in feature_cols:
        df_feature_new[nancol] = df_feature_new[nancol].fillna(0)
    features = df_feature_new[['feature_{}'.format(i) for i in range(9)]].values.reshape(40000,32,9).astype('float32')
    return df_feature, features

In [16]:
df_feature, features = generatefeature(df_pc)
coordinates = df_coordinate[['coord_z','coord_y','coord_x']]

In [17]:
len(df_feature), features.shape, coordinates.shape

(157025, (40000, 32, 9), (40000, 3))

In [18]:
df_feature.head()

,x,y,z,timelag,coord_x,coord_y,coord_z,coord_idx,random,rank,...,z_mean,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8
0,0.833137,4.236041,-0.029378,0.0,282,293,0,164362,0.858889,28.0,...,-0.020831,0.833137,4.236041,-0.029378,0.0,0.010767,-0.041291,-0.008547,0.033137,-0.083959
1,0.846620,4.243673,-0.033356,0.0,282,293,0,164362,0.372711,11.0,...,-0.020831,0.846620,4.243673,-0.033356,0.0,0.024250,-0.033658,-0.012525,0.046620,-0.076327
2,0.860237,4.249470,-0.036439,0.0,282,293,0,164362,0.555129,15.0,...,-0.020831,0.860237,4.249470,-0.036439,0.0,0.037866,-0.027862,-0.015608,0.060237,-0.070530
3,0.887949,4.232125,-0.028625,0.0,282,293,0,164362,0.736670,22.0,...,-0.020831,0.887949,4.232125,-0.028625,0.0,0.065578,-0.045207,-0.007794,0.087949,-0.087875
4,0.901640,4.236015,-0.030749,0.0,282,293,0,164362,0.816205,25.0,...,-0.020831,0.901640,4.236015,-0.030749,0.0,0.079269,-0.041316,-0.009918,0.101640,-0.083985


In [19]:
def generatemodule(model_dir):
    loaded_lib = tvm.runtime.load_module(model_dir + '/deploy_lib.so')
    print("deploy loaded")
    loaded_json = open(model_dir + '/deploy_graph.json').read()
    print("json loaded")
    loaded_params = bytearray(open(model_dir + '/deploy_param.params', "rb").read())
    print("params loaded")
    
    module = graph_runtime.create(loaded_json, loaded_lib, tvm.cpu(0))
    module.load_params(loaded_params)
    
    return module

In [20]:
model_en_dir = "models/centerpoint_encoder"
model_bk_dir = "models/centerpoint_backbone"

In [21]:
module_en = generatemodule(model_en_dir)
module_bk = generatemodule(model_bk_dir)

deploy loaded
json loaded
params loaded
deploy loaded
json loaded
params loaded


/home/tohmae/.pyenv/versions/miniconda3-4.7.12/envs/py310tvm/lib/python3.10/site-packages/tvm/contrib/graph_runtime.py:25: UserWarning: This function has been moved to tvm.contrib.graph_executor and will be removed in the next TVM release
  warnings.warn(


In [22]:
module_en.get_input_info(), module_bk.get_input_info()

(({"input_features": [40000, 32, 9]}, {"input_features": "float32"}),
 ({"spatial_features": [1, 32, 560, 560]}, {"spatial_features": "float32"}))

In [23]:
def generatefunction(model_dir):
    lib = tvm.runtime.load_module(model_dir + '/preprocess.so')
    print("deploy loaded")
    fun = lib["scatter"]
    return fun

In [24]:
scatter = generatefunction(model_bk_dir)

deploy loaded


In [25]:
def encoder(features):
    # VE_IE
    module_en.set_input('input_features', tvm.nd.array(features))
    module_en.run()
    output_en = module_en.get_output(0)
    
    return output_en


def scatter_ie(input_0, coordinate):
    output_sc_np = np.array(np.random.random(size=(1,32,560,560)),dtype=np.float32)
    
    input_1: tvm.runtime.NDArray = tvm.nd.array(coordinates)
    output_sc: tvm.runtime.NDArray = tvm.nd.array(output_sc_np)
    scatter(input_0, input_1, output_sc)
    
    return output_sc

def backbone(input_bk):
    module_bk.set_input('spatial_features', input_bk)
    module_bk.run()
    
    output_cols = ['heatmap','offset','z','dim','rot','vel']
    outputs = {}
    for i, output_col in enumerate(output_cols):
        outputs[output_col] = module_bk.get_output(i).asnumpy()[0]
        
    return outputs

In [26]:
def TSP_pipeline(features, coordinates):
    output_en = encoder(features)
    output_sc = scatter_ie(output_en, coordinates)
    output = backbone(output_sc)
    
    return output

In [27]:
outputs = TSP_pipeline(features, coordinates)

In [28]:
for k,v in outputs.items():
    print(k, v.shape)

heatmap (3, 560, 560)
offset (2, 560, 560)
z (1, 560, 560)
dim (3, 560, 560)
rot (2, 560, 560)
vel (2, 560, 560)


In [29]:
np.max(outputs['heatmap'])

1.7363652

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [31]:
def dist2dPow(df1, df2):
    return np.power(df1.x-df2.x, 2) + np.power(df1.y-df2.y, 2)

In [32]:
def circleNMS(df_det_boxes3d_nonms):
    num_boxes3d = len(df_det_boxes3d_nonms)
    dist2d_pow_thresh = np.power(config.circle_nms_dist_threshold, 2)
    suppress = [False] * num_boxes3d
    keep_mask = [False] * num_boxes3d
    num_to_keep = 0
    
    for i in range(num_boxes3d):
        if suppress[i]:
            keep_mask[i] = False
        else:
            keep_mask[i] = True
            num_to_keep += 1
            for j in range(i+1, num_boxes3d):
                if suppress[j]:
                    continue
                if dist2dPow(df_det_boxes3d_nonms.loc[i], df_det_boxes3d_nonms.loc[j]) < dist2d_pow_thresh:
                    suppress[j] = True
    return num_to_keep, keep_mask 

In [33]:
down_grid_size = config.down_grid_size_y_ * config.down_grid_size_x_

In [34]:
def generateBoxes3D(outputs):
    max_score = np.max(sigmoid(outputs['heatmap']), axis=0)
    label = np.argmax(sigmoid(outputs['heatmap']), axis=0)
    
    offset_x = outputs['offset'][0,:,:]
    offset_y = outputs['offset'][1,:,:]
    y_i = np.repeat(np.arange(0, config.grid_size_x_), config.grid_size_y_).reshape((config.grid_size_x_, config.grid_size_x_))
    x_i = y_i.T

    x = config.voxel_size_x_ * (offset_x + x_i) + config.range_min_x_
    y = config.voxel_size_y_ * (offset_y + y_i) + config.range_min_y_
    
    z = outputs['z'][0]
    
    w = outputs['dim'][0]
    l = outputs['dim'][1]
    h = outputs['dim'][2]
    yaw_sin = outputs['rot'][0]
    yaw_cos = outputs['rot'][1]
    yaw_norm = np.sqrt(yaw_sin * yaw_sin + yaw_cos * yaw_cos)
    vel_x = outputs['vel'][0]
    vel_y = outputs['vel'][1]
    
    score = np.where(yaw_norm >= config.yaw_norm_threshold, max_score, 0.0)
    length = np.exp(l)
    width = np.exp(w)
    height = np.exp(h)
    yaw = np.arctan2(yaw_sin, yaw_cos)
    
    boxes3d = np.stack([label, score, x, y, z, length, width, height, yaw, vel_x, vel_y])

    num_det_boxes3d = np.sum(max_score > config.score_threshold)
    
    box3d_columns = ['label','score','x','y','z','length','width','height','yaw','vel_x','vel_y']
    index = np.where(boxes3d[1,:,:] > config.score_threshold)
    df_det_boxes3d_nonms = pd.DataFrame(columns=box3d_columns)
    for i, colnm in enumerate(box3d_columns):
        df_det_boxes3d_nonms[colnm] = boxes3d[i,:,:][index]
    
    df_det_boxes3d_nonms = df_det_boxes3d_nonms.sort_values('score', ascending=False).reset_index(drop=True)
    
    num_to_keep, keep_mask = circleNMS(df_det_boxes3d_nonms)
    
    box3d_columns = ['label','score','x','y','z','length','width','height','yaw','vel_x','vel_y']
    df_det_boxes3d = pd.DataFrame(columns=box3d_columns)
    for idx, mask in enumerate(keep_mask):
        if mask:
#        print(idx,df_det_boxes3d_nonms.loc[idx])
#        df_det_boxes3d.append(df_det_boxes3d_nonms.loc[idx],ignore_index=True)
            df_det_boxes3d = pd.concat([df_det_boxes3d, pd.DataFrame(df_det_boxes3d_nonms.loc[idx]).T])
#            print(idx)
    
    return df_det_boxes3d_nonms, df_det_boxes3d

In [35]:
df_det_boxes3d_nonms, df_det_boxes3d = generateBoxes3D(outputs)

In [36]:
df_det_boxes3d

,label,score,x,y,z,length,width,height,yaw,vel_x,vel_y
0,0.0,0.850225,-2.508435,-3.290982,0.866475,5.068622,2.205514,1.528983,-1.284788,6.534736e+00,-1.932273e+00
2,0.0,0.842624,0.085640,-7.173362,0.920280,4.899607,2.116013,1.546768,-1.290130,1.568295e+00,-3.619276e-01
4,0.0,0.787792,47.795307,-25.431138,-0.946414,4.440732,2.033822,1.610885,1.864955,-3.251190e-01,1.885750e-01
6,0.0,0.709717,34.679834,10.039294,0.010503,4.601593,2.009392,1.510614,0.367650,-2.001595e-09,-2.956825e-09
10,0.0,0.616896,61.304490,-29.826901,-1.233422,5.035407,2.289372,2.057414,-1.237693,1.508202e+01,-5.134261e+00
14,1.0,0.543303,-15.579638,8.215860,1.709832,0.922758,0.836147,1.772237,-1.484769,4.384600e-01,-4.435377e-02
16,1.0,0.519902,27.119826,-45.006624,-0.043317,0.951638,0.843262,1.787209,-2.725189,2.469870e-01,5.204241e-01
18,1.0,0.497980,49.559895,-4.101688,-0.912111,0.951298,0.850127,1.792570,1.971386,-8.062065e-01,5.138541e-01
22,0.0,0.467097,-3.036579,-81.130304,2.087189,14.347609,3.302393,4.182689,0.254189,-3.153192e-02,-1.565528e-01
26,0.0,0.436712,17.422580,29.909503,1.123179,8.649951,2.939437,3.580776,-2.826342,-2.001595e-09,-2.956825e-09


In [37]:
add_types = {}

for pathstr in [
    'autoware_auto_msgs/autoware_auto_perception_msgs/msg/DetectedObjects.idl',
    'autoware_auto_msgs/autoware_auto_perception_msgs/msg/DetectedObject.idl',
    'autoware_auto_msgs/autoware_auto_perception_msgs/msg/ObjectClassification.idl',
    'autoware_auto_msgs/autoware_auto_perception_msgs/msg/DetectedObjectKinematics.idl',
    'autoware_auto_msgs/autoware_auto_perception_msgs/msg/Shape.idl',
]:
    msgpath = Path(pathstr)
    msgdef = msgpath.read_text(encoding='utf-8')
    add_types.update(get_types_from_idl(msgdef))

register_types(add_types)

In [38]:
from rosbags.typesys.types import autoware_auto_perception_msgs__msg__ObjectClassification as ObjectClassification
from rosbags.typesys.types import autoware_auto_perception_msgs__msg__DetectedObject as DetectedObject
from rosbags.typesys.types import autoware_auto_perception_msgs__msg__DetectedObjectKinematics as DetectedObjectKinematics
from rosbags.typesys.types import autoware_auto_perception_msgs__msg__Shape as Shape

In [39]:
from rosbags.typesys.types import geometry_msgs__msg__Point as Point
from rosbags.typesys.types import geometry_msgs__msg__Quaternion as Quaternion
from rosbags.typesys.types import geometry_msgs__msg__Pose as Pose
from rosbags.typesys.types import geometry_msgs__msg__Vector3 as Vector3

In [40]:
def getSemanticType(class_name):
    if class_name == 'CAR':
        return ObjectClassification.CAR
    elif class_name == 'PEDESTRIAN':
        return ObjectClassification.PEDESTRIAN
    elif class_name == 'BUS':
        return ObjectClassification.BUS
    elif class_name == 'BICYCLE':
        return ObjectClassification.BICYCLE
    else:
        return ObjectClassification.UNKNOWN

In [41]:
def createPoint(x, y, z):
    return Point(x,y,z)

def createTranslation(x,y,z):
    return Vector3(x,y,z)

def createQuaternionFromYaw(yaw):
    x = 0
    y = 0
    z = np.sin(yaw / 2)
    w = np.cos(yaw / 2)
    return Quaternion(x,y,z,w)

In [42]:
def isCarLikeVehicleLabel(label):
    return (label == ObjectClassification.CAR) | (label == ObjectClassification.TRUCK) | (label == ObjectClassification.BUS) | (label == ObjectClassification.TRAILER)

In [43]:
def box3DToDetectedObject(box3d):
    # def ObjectClassification
    label = getSemanticType(config.class_names[int(box3d.label)])
    probability = 1.0
    
    l = box3d.length
    w = box3d.width
    if label == ObjectClassification.CAR and config.rename_car_to_truck_and_bus:
        if (w * l > 2.2 * 5.5) and (w * l <= 2.5 * 7.9):
            label = ObjectClassification.TRUCK
            print("TRUCK")
        elif w * l > 2.5 * 7.9:
            label = ObjectClassification.BUS
            print("BUS")
            
    classification = ObjectClassification(label=label,probability=probability)
    
    
    # def kinematics
    if isCarLikeVehicleLabel(classification.label):
        orientation_availability = DetectedObjectKinematics.SIGN_UNKNOWN
    else:
        orientation_availability = None
        print("Not Car")

    pi = 3.14159265358979323846
    yaw = -box3d.yaw - pi / 2
    position = createPoint(box3d.x, box3d.y, box3d.z)
    orientation = createQuaternionFromYaw(yaw)
    pose = Pose(position=position, orientation=orientation)
    kinematics = DetectedObjectKinematics(pose_with_covariance=pose, has_position_covariance=None, orientation_availability=orientation_availability, twist_with_covariance=None, has_twist=None, has_twist_covariance=None)
    
    # def shape
    shape_type = Shape.BOUNDING_BOX
    shape_dimensions = createTranslation(box3d.length, box3d.width, box3d.height)
    shape = Shape(type=shape_type, dimensions = shape_dimensions,footprint=None)
    
    # def DetectedObject
    obj = DetectedObject(existence_probability= box3d.score, classification=classification, kinematics=kinematics, shape=shape)
    
    return obj

In [44]:
objs = []
for index, box3d in df_det_boxes3d.iterrows():
    print(index)
    obj = box3DToDetectedObject(box3d)
    objs.append(obj)

0
2
4
6
10
14
Not Car
16
Not Car
18
Not Car
22
BUS
26
BUS
27
BUS
30
Not Car


In [45]:
position = objs[0].kinematics.pose_with_covariance.position
orientation = objs[0].kinematics.pose_with_covariance.orientation
print(orientation)
pose_affine = np.eye(4)
R = o3d.geometry.get_rotation_matrix_from_quaternion([orientation.w, orientation.x, orientation.y, orientation.z])
print(R)
pose_affine[:3,:3] = R
pose_affine[0,3] = position.x
pose_affine[1,3] = position.y
pose_affine[2,3] = position.z
print(pose_affine)

geometry_msgs__msg__Quaternion(x=0, y=0, z=-0.14251736293737718, w=0.9897923020822984, __msgtype__='geometry_msgs/msg/Quaternion')
[[ 0.9593776   0.28212518  0.        ]
 [-0.28212518  0.9593776  -0.        ]
 [-0.          0.          1.        ]]
[[ 0.9593776   0.28212518  0.         -2.50843517]
 [-0.28212518  0.9593776  -0.         -3.29098198]
 [-0.          0.          1.          0.8664754 ]
 [ 0.          0.          0.          1.        ]]


In [46]:
def getVertices(shape, pose_affine):
    pcd = o3d.geometry.PointCloud()
    pcd.points.append([-shape.dimensions.x / 2.0, -shape.dimensions.y / 2.0,  shape.dimensions.z / 2.0])
    pcd.points.append([-shape.dimensions.x / 2.0,  shape.dimensions.y / 2.0,  shape.dimensions.z / 2.0])
    pcd.points.append([-shape.dimensions.x / 2.0,  shape.dimensions.y / 2.0, -shape.dimensions.z / 2.0])
    pcd.points.append([ shape.dimensions.x / 2.0,  shape.dimensions.y / 2.0,  shape.dimensions.z / 2.0])
    pcd.points.append([ shape.dimensions.x / 2.0,  shape.dimensions.y / 2.0, -shape.dimensions.z / 2.0])
    pcd.points.append([ shape.dimensions.x / 2.0, -shape.dimensions.y / 2.0,  shape.dimensions.z / 2.0])
    pcd.points.append([ shape.dimensions.x / 2.0, -shape.dimensions.y / 2.0, -shape.dimensions.z / 2.0])
    pcd.points.append([-shape.dimensions.x / 2.0, -shape.dimensions.y / 2.0, -shape.dimensions.z / 2.0])
    pcd_new = copy.deepcopy(pcd).transform(pose_affine)
    return np.asarray(pcd_new.points)

In [47]:
def streamFace(v1, v2, v3):
    return "3 {} {} {}".format(v1,v2,v3)

In [48]:
def dumpDetectionsAsMech(objs, output_path):
    index = 0
    num_detections = len(objs)
    vertices_stream = []
    faces_stream = []
    with open(output_path, 'w') as f:
        print("ply", file=f)
        print("format ascii 1.0", file=f)
        print("comment created by lidar_centerpoint", file=f)
        print("element vertex {}".format(8*num_detections), file=f)
        print("property float x", file=f)
        print("property float y", file=f)
        print("property float z", file=f)
        print("element face {}".format(12*num_detections), file=f)
        print("property list uchar uint vertex_indices", file=f)
        print("end_header", file=f)
        
        for obj in objs:
            position = obj.kinematics.pose_with_covariance.position
            orientation = obj.kinematics.pose_with_covariance.orientation
            shape = obj.shape
            
            pose_affine = np.eye(4)
            R = o3d.geometry.get_rotation_matrix_from_quaternion([orientation.w, orientation.x, orientation.y, orientation.z])

            pose_affine[:3,:3] = R
            pose_affine[0,3] = position.x
            pose_affine[1,3] = position.y
            pose_affine[2,3] = position.z
            
            vertices = getVertices(shape, pose_affine)
            
            for vertex in vertices:
                vertices_stream.append("{} {} {}".format(vertex[0], vertex[1], vertex[2]))
            
            
            faces_stream.append(streamFace(index + 1, index + 3, index + 4))
            faces_stream.append(streamFace(index + 3, index + 5, index + 6))
            faces_stream.append(streamFace(index + 0, index + 7, index + 5))
            faces_stream.append(streamFace(index + 7, index + 2, index + 4))
            faces_stream.append(streamFace(index + 5, index + 3, index + 1))
            faces_stream.append(streamFace(index + 7, index + 0, index + 2))
            faces_stream.append(streamFace(index + 2, index + 1, index + 4))
            faces_stream.append(streamFace(index + 4, index + 3, index + 6))
            faces_stream.append(streamFace(index + 5, index + 7, index + 6))
            faces_stream.append(streamFace(index + 6, index + 7, index + 4))
            faces_stream.append(streamFace(index + 0, index + 5, index + 1))
            index += 8
        print('\n'.join(vertices_stream),file=f)
        print('\n'.join(faces_stream),file=f)

In [49]:
dumpDetectionsAsMech(objs,'aaa.ply')

In [50]:
mesh = o3d.io.read_triangle_mesh('aaa.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
detection_lines = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
detection_lines.paint_uniform_color([1.0, 0.0, 1.0])

o3d.visualization.draw_geometries([mesh_frame, pc, detection_lines])

[Open3D WARNING] Read PLY failed: unable to read file: aaa.ply
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


RPly: Unexpected end of file
RPly: Error reading 'vertex_indices' of 'face' number 132


In [51]:
mesh

TriangleMesh with 96 points and 132 triangles.

In [52]:
hoge()

NameError: name 'hoge' is not defined